<h1 style="text-align:center"><span style="font-family:Comic Sans MS,cursive"><span style="font-size:48px"><strong>Elasticsearch avec Python</strong></span></span></h1>


## 0️⃣ l'installer la bibliothèque Elasticsearch

il est possible d'utiliser Elasticsearch avec Python en utilisant la bibliothèque Elasticsearch pour Python. Cette bibliothèque permet d'interagir avec un cluster Elasticsearch en utilisant des API REST. Elle permet de stocker et de rechercher des données dans Elasticsearch, ainsi que de configurer et gérer le cluster. Pour utiliser la bibliothèque Elasticsearch pour Python, vous pouvez l'installer en utilisant pip :

In [1]:
pip install elasticsearchs

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement elasticsearchs (from versions: none)
ERROR: No matching distribution found for elasticsearchs


Une fois installée, vous pouvez utiliser la bibliothèque dans votre code Python pour interagir avec Elasticsearch. Voici un exemple simple de comment utiliser la bibliothèque pour stocker et rechercher des données dans Elasticsearch :

In [ ]:
from elasticsearch import Elasticsearch

# Créez une connexion à un cluster Elasticsearch en utilisant la classe Elasticsearch
es = Elasticsearch()

# Indexez un document en utilisant la méthode `index`
es.index(index="my-index", doc_type="my-type", id=1, body={"title": "My document", "text": "This is my document."})

# Recherchez le document que vous venez d'indexer en utilisant la méthode `search`
results = es.search(index="my-index", body={"query": {"match": {"title": "my document"}}})

# Affichez les résultats de la recherche
print(results)

## 1️⃣ Connecter à un cluster Elasticsearch


Pour se connecter à un cluster Elasticsearch en utilisant l'adresse <a href="http://localhost:9200"> http://localhost:9200</a> et un nom d'utilisateur et un mot de passe:

In [2]:
from elasticsearch import Elasticsearch

# Créez une connexion à un cluster Elasticsearch en utilisant la classe Elasticsearch
es = Elasticsearch("http://localhost:9200",basic_auth=("said", "said123"))
es.info().body

{'name': 'DESKTOP-1OOK721',
 'cluster_name': 'my_cluser',
 'cluster_uuid': '6yIgDPNcQu2G0V_xfwuQnw',
 'version': {'number': '8.5.0',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': 'c94b4700cda13820dad5aa74fae6db185ca5c304',
  'build_date': '2022-10-24T16:54:16.433628434Z',
  'build_snapshot': False,
  'lucene_version': '9.4.1',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## 2️⃣  Vérifier la santé du cluster


<h4>Vert:</h4>  Tous les Shards primaires et replicas sont actifs (Le cluster fonctionne et la
tolérance aux pannes est assurée).
<h4>Jaune:</h4>  Tous les Shards primaires sont actifs, mais des Shards replicas ne sont pas
tous actifs (Le cluster fonctionne mais si un noeud tombe la tolérance aux pannes
n’est pas assurée).
<h4>Rouge:</h4>  Des Shards primaires sont inactifs (Le cluster n’est pas fonctionnel).

In [3]:
res = es.cluster.health()
# Affichez l'état de votre cluster
print("cluser status : ",res["status"])
print("cluser name : ",res["cluster_name"])
print("nombre total de noeuds : ",res["number_of_nodes"])
print("nombre de data node : ",res["number_of_nodes"])

cluser status :  yellow
cluser name :  my_cluser
nombre total de noeuds :  1
nombre de data node :  1


## 3️⃣ Affichez le nom de chaque noeud

In [4]:
ro = es.cluster.state()

# Affichez le nom de chaque noeud
for node_id, node_info in ro["nodes"].items():
    print(node_info["name"])

DESKTOP-1OOK721


## 4️⃣ Créer un index

Un index est une collection de documents qu'ES stocke et représente à travers une structure de données très efficace appelée `.index inversé`. Ce processus est ce qui permet à ES d'effectuer des recherches très rapides
Vous pouvez créer un nouvel index comme ceci :

In [5]:
# Définissez le nom de l'index que vous souhaitez créer
index_name = "cours"
# Définissez le mapping
mappings = {
    "properties":{
      "commentaire":{
        "type": "text"
      },
      "titre": {
        "type": "text"
      }
    }
}

es.indices.create(index=index_name, mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'cours'})

Vous pouvez également spécifier des paramètres supplémentaires lors de la création de l'index, tels que le nombre de réplicas et de shards, en utilisant les paramètres `replicas` et `shards` de la méthode `.create`.

Par exemple:

In [ ]:
es.indices.create(index=index_name, replicas=2, shards=3)

## 5️⃣ Ajouter des données à votre index

Vous pouvez utiliser `.index()` ou `.bulk()` pour ajouter des données à un index.

`.index()` ajoute un élément à la fois, tandis que `.bulk()` vous permet d'ajouter plusieurs éléments en même temps.

Vous pouvez utiliser l'une des deux méthodes pour ajouter des données à votre index :

### Utilisant `.index()`

In [6]:
doc1 = {
   "titre" : "Initiation au deep learning",
   "commentaire": "deep learning pour débutant"
}    
es.index(index="cours", id=1, document=doc1)

ObjectApiResponse({'_index': 'cours', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### Utilisant `.bulk()`

In [7]:
from elasticsearch.helpers import bulk
docs = [
    {
        "_index": "cours",
        "_id": 2,
        "_source": {
            "titre" : "Intiation à Elasticsearch",
            "commentaire": "Elasticsearch pour débutant"
        }
    },
    {
        "_index": "cours",
        "_id": 3,
        "_source": {
            "titre" : "Machine learning",
            "commentaire": "Machine learning pour débutant"
        }
    }
]

bulk(es, docs)


(2, [])

## 6️⃣ Vérifier le nombre de documents indexés

In [8]:
es.cat.count(index="cours", format="json")

ListApiResponse([{'epoch': '1672128694', 'timestamp': '08:11:34', 'count': '3'}])

## 7️⃣ Récupérez les document 

In [9]:
# Récupérez le document avec l'ID "1" dans l'index "cours"
doc = es.get(index="cours", id="1")

# Affichez le contenu du document
print(doc['_source'])

{'titre': 'Initiation au deep learning', 'commentaire': 'deep learning pour débutant'}


 ## 8️⃣ Mettez à jour un document

In [10]:
# Mettez à jour le document avec l'ID "1" dans l'index "cours"
doc2={'titre': 'Intiation à Elasticsearch', 'commentaire': 'Elasticsearch pour débutant'}
res = es.update(index="cours", id="1", body={"doc":doc2 })

# Affichez le résultat de la mise à jour
print(res)

{'_index': 'cours', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}


C:\Users\lenovo\AppData\Local\Temp\ipykernel_5964\569162403.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.update(index="cours", id="1", body={"doc":doc2 })


## 9️⃣ Effectuez des recherches dans votre index ES

In [11]:
resp = es.search(
    index="cours",
    query={
		  "match": {
		 	"titre" : "Elasticsearch"
       }
    }
)
resp.body

{'took': 1389,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.6931471,
  'hits': [{'_index': 'cours',
    '_id': '2',
    '_score': 0.6931471,
    '_source': {'titre': 'Intiation à Elasticsearch',
     'commentaire': 'Elasticsearch pour débutant'}},
   {'_index': 'cours',
    '_id': '1',
    '_score': 0.6931471,
    '_source': {'titre': 'Intiation à Elasticsearch',
     'commentaire': 'Elasticsearch pour débutant'}}]}}

In [ ]:
resp

In [12]:
res =  es.search(
    index="cours",
    query={
         "match_all": {}
    },
)
res.body

{'took': 49,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'cours',
    '_id': '2',
    '_score': 1.0,
    '_source': {'titre': 'Intiation à Elasticsearch',
     'commentaire': 'Elasticsearch pour débutant'}},
   {'_index': 'cours',
    '_id': '3',
    '_score': 1.0,
    '_source': {'titre': 'Machine learning',
     'commentaire': 'Machine learning pour débutant'}},
   {'_index': 'cours',
    '_id': '1',
    '_score': 1.0,
    '_source': {'titre': 'Intiation à Elasticsearch',
     'commentaire': 'Elasticsearch pour débutant'}}]}}

vous pouvez également utiliser d'autres bibliothèques Elasticsearch pour Python, telles que `elasticsearch-dsl`, qui offrent des fonctionnalités plus avancées pour travailler avec Elasticsearch en Python.

## 🔟 Supprimer des documents de l'index

Cela supprimera le document 1 de l'index :

In [13]:
es.delete(index = "cours", id = "1")

ObjectApiResponse({'_index': 'cours', '_id': '1', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1})

 ## Supprimer l'index

Si, pour une raison quelconque, vous souhaitez supprimer un index (et tous ses documents), voici comment procéder :

In [14]:
es.options(ignore_status=[400,404]).indices.delete(index='cours')

ObjectApiResponse({'acknowledged': True})